<a href="https://colab.research.google.com/github/satani99/prosody_clone/blob/main/Toucan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/satani99/IMS-Toucan.git

Cloning into 'IMS-Toucan'...
remote: Enumerating objects: 16665, done.
remote: Counting objects: 100% (5441/5441), done.
remote: Compressing objects: 100% (1177/1177), done.
remote: Total 16665 (delta 4380), reused 4264 (delta 4264), pack-reused 11224
Receiving objects: 100% (16665/16665), 11.87 MiB | 13.40 MiB/s, done.
Resolving deltas: 100% (12591/12591), done.


In [2]:
%cd IMS-Toucan

/content/IMS-Toucan


In [3]:
!pip install alias_free_torch auraloss cvxopt deepfilternet dragonmapper gradio jinja2 librosa matplotlib==3.4.3 numba numpy pedalboard phonemizer praat-parselmouth pyloudnorm pypinyin scikit-learn scipy sounddevice soundfile speechbrain torch torchaudio torch_complex tqdm urllib3 wandb six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.9/37.9 MB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.0/519.0 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.1/544.1 kB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 7.2 MB/s eta 0:00:

In [4]:
import os
import numpy as np
import soundfile
import soundfile as sf
import torch
from tqdm import tqdm

In [5]:
!sudo apt-get install espeak-ng
!sudo apt-get install libportaudio2

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
The following NEW packages will be installed:
  espeak-ng espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 8 not upgraded.
Need to get 4,525 kB of archives.
After this operation, 11.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpcaudio0 amd64 1.1-6build2 [8,956 B]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsonic0 amd64 0.2.0-11build1 [10.3 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 espeak-ng-data amd64 1.50+dfsg-10 [3,956 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 libespeak-ng1 amd64 1.50+dfsg-10 [207 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 espeak-ng amd64 1.50+dfsg-10 [343 kB]
Fetched 4,525 kB in 2s (1,958 kB

In [6]:
from run_model_downloader import download_models
download_models()

[..................................................][██................................................][███...............................................][█████.............................................][███████...........................................][█████████.........................................][███████████.......................................][█████████████.....................................][███████████████...................................][███████████

In [7]:
from Preprocessing.TextFrontend import ArticulatoryCombinedTextFrontend
from Preprocessing.AudioPreprocessor import AudioPreprocessor
from TrainingInterfaces.Text_to_Spectrogram.AutoAligner.Aligner import Aligner
from TrainingInterfaces.Text_to_Spectrogram.FastSpeech2.DurationCalculator import DurationCalculator
from InferenceInterfaces.UtteranceCloner import UtteranceCloner
from Preprocessing.articulatory_features import get_feature_to_index_lookup

In [8]:
def float2pcm(sig, dtype="int16"):
  sig = np.asarray(sig)
  if sig.dtype.kind != 'f':
    raise TypeError("'sig' must be a float array")
  dtype = np.dtype(dtype)
  dtype = np.dtype(dtype)
  if dtype.kind not in 'iu':
    raise TypeError("'dtype' must be an integer type")
  i = np.iinfo(dtype)
  abs_max = 2 ** (i.bits - 1)
  offset = i.min + abs_max
  return (sig * abs_max + offset).clip(i.min, i.max).astype(dtype)
